In [20]:
#Import Libraries

import os
import glob
import datetime

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import h5py
import json
import time

In [21]:
config={
  "train_path"      : "/Users/rishitdholakia/Desktop/Car Damage Detection project/data3a/training",
  "test_path"       : "/Users/rishitdholakia/Desktop/Car Damage Detection project/data3a/validation",
  "features_path"   : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Severity/features.h5",
  "labels_path"     : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Severity/labels.h5",
  "model_path"      : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Car Damage check/model0.2.json",
  "weights_path"    : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Car Damage check/model0.2.h5",
  "classifier_path" : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Severity/classifier.pickle", 
  "test_size"       : 0.20,
  "seed"            : 9,
  "image_size"      : (224, 224)
}

In [22]:
#load saved model
json_file = open(config["model_path"], 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [23]:
#load saved weights
loaded_model.load_weights(config["weights_path"])

In [24]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [25]:
# config variables
train_path    = config["train_path"]
features_path = config["features_path"]
labels_path   = config["labels_path"]
image_size    = config["image_size"]
seed          = config["seed"]
classifier_path = config["classifier_path"]
test_size = config["test_size"]

In [8]:
# path to training dataset
train_labels = os.listdir(train_path)

# encode the labels
print ("Encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = []
labels   = []

# loop over all the labels in the folder
count = 1
for i, label in enumerate(train_labels):
    cur_path = train_path + "/" + label
    count = 1
    for image_path in glob.glob(cur_path + "/*.jpg"):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = loaded_model.predict(x)
        flat = feature.flatten()
        features.append(flat)
        labels.append(label)
        print ("Processed - " + str(count))
        count += 1
    print ("Completed label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print ("[STATUS] training labels: {}".format(le_labels))
print ("[STATUS] training labels shape: {}".format(le_labels.shape))

Encoding labels...
Completed label - .DS_Store
Processed - 1
Processed - 2
Processed - 3
Processed - 4
Processed - 5
Processed - 6
Processed - 7
Processed - 8
Processed - 9
Processed - 10
Processed - 11
Processed - 12
Processed - 13
Processed - 14
Processed - 15
Processed - 16
Processed - 17
Processed - 18
Processed - 19
Processed - 20
Processed - 21
Processed - 22
Processed - 23
Processed - 24
Processed - 25
Processed - 26
Processed - 27
Processed - 28
Processed - 29
Processed - 30
Processed - 31
Processed - 32
Processed - 33
Processed - 34
Processed - 35
Processed - 36
Processed - 37
Processed - 38
Processed - 39
Processed - 40
Processed - 41
Processed - 42
Processed - 43
Processed - 44
Processed - 45
Processed - 46
Processed - 47
Processed - 48
Processed - 49
Processed - 50
Processed - 51
Processed - 52
Processed - 53
Processed - 54
Processed - 55
Processed - 56
Processed - 57
Processed - 58
Processed - 59
Processed - 60
Processed - 61
Processed - 62
Processed - 63
Processed - 64
Pr

Processed - 208
Processed - 209
Processed - 210
Processed - 211
Processed - 212
Processed - 213
Processed - 214
Processed - 215
Processed - 216
Processed - 217
Processed - 218
Processed - 219
Processed - 220
Processed - 221
Processed - 222
Processed - 223
Processed - 224
Processed - 225
Processed - 226
Processed - 227
Processed - 228
Processed - 229
Processed - 230
Processed - 231
Processed - 232
Processed - 233
Processed - 234
Processed - 235
Processed - 236
Processed - 237
Processed - 238
Processed - 239
Processed - 240
Processed - 241
Processed - 242
Processed - 243
Processed - 244
Processed - 245
Processed - 246
Processed - 247
Processed - 248
Processed - 249
Processed - 250
Processed - 251
Processed - 252
Processed - 253
Processed - 254
Processed - 255
Processed - 256
Processed - 257
Processed - 258
Processed - 259
Processed - 260
Processed - 261
Processed - 262
Processed - 263
Processed - 264
Processed - 265
Processed - 266
Processed - 267
Processed - 268
Processed - 269
Processe

In [9]:
# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# end time
end = time.time()
print ("End time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

End time - 2021-03-24 00:53


In [26]:
# imports required packages

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [27]:
features=np.array(features)
labels=np.array(labels)

In [56]:
# verify the shape of features and labels
print ("[INFO] features shape: {}".format(features.shape))
print ("[INFO] labels shape: {}".format(labels.shape))

print ("[INFO] training started...")
# split the training and testing data
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(features),
                                                                  np.array(le_labels),
                                                                  test_size=test_size,
                                                                  random_state=seed)

print ("[INFO] splitted train and test data...")
print ("[INFO] train data  : {}".format(trainData.shape))
print ("[INFO] test data   : {}".format(testData.shape))
print ("[INFO] train labels: {}".format(trainLabels.shape))
print ("[INFO] test labels : {}".format(testLabels.shape))

[INFO] features shape: (979, 4096)
[INFO] labels shape: (979,)
[INFO] training started...
[INFO] splitted train and test data...
[INFO] train data  : (783, 4096)
[INFO] test data   : (196, 4096)
[INFO] train labels: (783,)
[INFO] test labels : (196,)


In [57]:
# use logistic regression as the model
print ("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(trainData, trainLabels)

[INFO] creating model...


/anaconda3/envs/newenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=9)

In [58]:
# evaluate the model of test data
preds = model.predict(testData)

In [59]:
# plot the confusion matrix
cm = confusion_matrix(testLabels, preds)

In [60]:
# dump classifier to file
print ("[INFO] saving model...")
pickle.dump(model, open(classifier_path, 'wb'))

# display the confusion matrix
print ("[INFO] confusion matrix")

# plot the confusion matrix
cm = confusion_matrix(testLabels, preds)

score = model.score(testData, testLabels)
print(score)
score = model.score(trainData, trainLabels)
print(score)

[INFO] saving model...
[INFO] confusion matrix
0.6887755102040817
1.0


In [66]:
#read in the image, pre-proces the image and make predictions
def check_severity(imagepath):
    ans=""
    img = image.load_img(imagepath, target_size=config["image_size"])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = loaded_model.predict(x)
    preds = model.predict(features)
    if preds==0:
        ans="Minor"
    elif (preds==1):
        ans="Moderate"
    else:
        ans="Severe"
    return ans

In [67]:
pred=check_severity('/Users/rishitdholakia/Desktop/Car Damage Detection project/data3a/validation/01-minor/5.jpg')
print(pred)

Minor
